## Model

In [1]:
# from asr_model import ASRModel

# a, b, c = ASRModel(model_dim=768, mode = 'A').to('cuda'), ASRModel(model_dim=768, mode = 'B').to('cuda'), ASRModel(model_dim=768, mode = 'C').to('cuda')
# a.params, b.params, c.params

from asr_model import ASRModel

a_last6 = ASRModel(model_dim=768, mode = 'A', layer_selection_mode='last6').to('cuda')
a_last3_first3 = ASRModel(model_dim=768, mode = 'A', layer_selection_mode='first3_last3').to('cuda')
# b = ASRModel(model_dim=768, mode = 'B').to('cuda')
a_last6.params, a_last3_first3.params

c:\Users\tuleh\anaconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\tuleh\anaconda3\envs\myenv\Lib\site-packages\torch\nn\modules\transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


(168997376, 168997760)

In [2]:
import torch
audio_features = torch.randn(10, 20, 10240).to('cuda')  # Example input tensor (seq_len, batch_size, acoustic_input_dim)
input_ids = torch.randint(0, a_last6.vocab_size, (10, 35)).long().to('cuda')
attention_mask = torch.ones(10, 35).to('cuda')  # Example attention mask (batch_size, seq_len)
input_ids.shape, audio_features.shape, attention_mask.shape

(torch.Size([10, 35]), torch.Size([10, 20, 10240]), torch.Size([10, 35]))

In [6]:
a_lala = a_last6(input_ids = input_ids,
           attention_mask = attention_mask,
           audio_features = audio_features)
print(a_lala.shape)

torch.Size([10, 35, 32128])


In [7]:
a_lele = a_last3_first3(input_ids = input_ids,
           attention_mask = attention_mask,
           audio_features = audio_features)
print(a_lele.shape)

torch.Size([10, 35, 32128])


In [5]:
hihi = a_lala.argmax(dim=-1)
hihi

tensor([[18709, 24352, 27565,  1301, 27287, 27853, 19539, 14865,  1713,  1098,
         25399, 27336,  8245,  6407,  5206, 17804, 23071, 28113,  9647, 22601,
         10010, 20818, 29141, 16834, 15952, 12004, 13890, 15369, 16810, 23345,
          6498,  5475, 28467, 19619, 12016],
        [27062, 14089,  4540, 25402,  8996, 18154,  2325, 27679,   793,  1259,
         12432, 17898,  7090, 18738, 26590,  4858, 13563, 26742, 14417,  7151,
         17575, 25988, 26449,  2034,  7990,  6789,  3799, 27871,  8362,  3642,
         21410, 24595, 16655, 10161, 14096],
        [19218, 27976, 23676, 26617,  5807,  8588, 28033, 11417,  3006,  4410,
         20807, 30994, 17011,  7036, 10534, 16325,  7222, 27857, 27184, 14556,
          4415,  3076, 15633,  8371, 27896, 17479,  5869, 28508, 23160, 10998,
         22800, 22789, 25967,  5484, 11911],
        [28865,  5025, 10183, 27720, 18769, 27122, 27683, 23721,  6896, 29061,
          1889, 21086, 11837, 21362,  8240, 28411,  5730, 28528, 19615, 208

In [9]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
a[:-6], a[-6:]

([1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12])

In [4]:
b_lala = b(input_ids = input_ids,
           attention_mask = None,
           audio_features = audio_features)
print(b_lala.shape)

torch.Size([10, 35, 32128])


In [7]:
c_lala = c(input_ids = input_ids,
           attention_mask = None,
           audio_features = audio_features)
print(c_lala.shape)

torch.Size([10, 35, 21128])


In [9]:
a_last6.decoder._shift_right

AttributeError: 'Decoder' object has no attribute '_shift_right'

# Train Function

In [10]:
def _shift_right(input_ids, decoder_start_token_id, pad_token_id):
    """
    Shifts the input_ids to the right and prepends the decoder_start_token_id.

    Args:
        input_ids (torch.Tensor): Tensor of shape (batch_size, seq_length) containing input token IDs.
        decoder_start_token_id (int): The token ID to prepend at the start of each sequence.
        pad_token_id (int): The token ID used for padding.

    Returns:
        torch.Tensor: Tensor of shape (batch_size, seq_length) with shifted input IDs.
    """
    if decoder_start_token_id is None:
        raise ValueError(
            "decoder_start_token_id must be defined. In T5, it is usually set to the pad_token_id."
        )

    if pad_token_id is None:
        raise ValueError("pad_token_id must be defined.")

    # Create a tensor for the shifted input IDs
    shifted_input_ids = input_ids.new_zeros(input_ids.shape)

    # Shift inputs to the right
    shifted_input_ids[..., 1:] = input_ids[..., :-1].clone()

    # Replace the first token with the decoder_start_token_id
    shifted_input_ids[..., 0] = decoder_start_token_id

    # Replace possible -100 values in labels with `pad_token_id`
    shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)

    return shifted_input_ids

import torch
from transformers import T5Tokenizer
from asr_model import ASRModel
from evaluate import load
cer = load("cer")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained("Langboat/mengzi-t5-base")
a_last6 = ASRModel(model_dim=768, mode = 'A', layer_selection_mode='last6').to('cuda')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
c:\Users\tuleh\anaconda3\envs\myenv\Lib\site-packages\torch\nn\modules\transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [11]:
# Tokenzier
# bos_token = tokenizer.cls_token_id
# eos_token = tokenizer.sep_token_id
# pad_token = tokenizer.pad_token_id

# Example text input and features
text = ["而 对 楼市 成交 抑制 作用 最 大 的 限 购", "也 成为 地方 政府 的 眼中 钉", "自 六月 底 呼和浩特 市 率先 宣布 取消 限 购 后", "各地 政府 便 纷纷 跟进"]
input_ids = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length = 256)
decoded_text = tokenizer.batch_decode(input_ids["input_ids"])
decoder_start_token_id = a_last6.decoder.config.decoder_start_token_id
shifted_input_ids = _shift_right(input_ids["input_ids"], decoder_start_token_id, tokenizer.pad_token_id)
shifted_text = tokenizer.batch_decode(shifted_input_ids)
print(f'Original text: {text}')
print(f'Text used as ground-truth: {decoded_text}')
print(f'Text used as training data: {shifted_text}')

downsampled_features = torch.rand(4, 128, 205, 80) # (B, C, T, F) -> (B, T, C*F)
batch = {
    'input_ids': input_ids.input_ids,
    'attention_mask': input_ids.attention_mask,
    'downsampled_features': downsampled_features,
}
for k in batch:
    batch[k] = batch[k].to(device=device, non_blocking=True)

Original text: ['而 对 楼市 成交 抑制 作用 最 大 的 限 购', '也 成为 地方 政府 的 眼中 钉', '自 六月 底 呼和浩特 市 率先 宣布 取消 限 购 后', '各地 政府 便 纷纷 跟进']
Text used as ground-truth: ['而 对 楼市 成交 抑制 作用 最 大 的 限 购</s>', '也 成为 地方 政府 的 眼中 钉</s><pad><pad><pad><pad><pad><pad><pad><pad>', '自 六月 底 呼和浩特 市 率先 宣布 取消 限 购 后</s>', '各地 政府 便 纷纷 跟进</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>']
Text used as training data: ['<pad>而 对 楼市 成交 抑制 作用 最 大 的 限 购', '<pad> 也 成为 地方 政府 的 眼中 钉</s><pad><pad><pad><pad><pad><pad><pad>', '<pad>自 六月 底 呼和浩特 市 率先 宣布 取消 限 购 后', '<pad> 各地 政府 便 纷纷 跟进</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>']


In [13]:
input_ids["input_ids"][1]

tensor([  34,    7,  263,    7,  967,    7,  619,    7,    5,    7, 7315,    7,
        8741,    1,    0,    0,    0,    0,    0,    0,    0,    0])

In [14]:
input_ids["attention_mask"][1]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [23]:
input_ids.input_ids[:2]

tensor([[  36,    7,   27,    7, 8169,    7, 3583,    7, 4915,    7,  759,    7,
          126,    7,   28,    7,    5,    7, 4020,    7, 3572,    1],
        [  34,    7,  263,    7,  967,    7,  619,    7,    5,    7, 7315,    7,
         8741,    1,    0,    0,    0,    0,    0,    0,    0,    0]])

In [25]:
tokenizer.batch_decode(shifted_input_ids)

['<pad>而 对 楼市 成交 抑制 作用 最 大 的 限 购',
 '<pad> 也 成为 地方 政府 的 眼中 钉<pad><pad><pad><pad><pad><pad><pad><pad>',
 '<pad>自 六月 底 呼和浩特 市 率先 宣布 取消 限 购 后',
 '<pad> 各地 政府 便 纷纷 跟进<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>']

In [15]:
tokenizer.all_special_ids[:3], tokenizer.all_special_tokens[:3]

([1, 2, 0], ['</s>', '<unk>', '<pad>'])

In [6]:
a_last6.decoder.config

T5Config {
  "_attn_implementation_autoset": true,
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.2",
  "use_cache": true,
  "vocab_size": 32128
}

In [25]:
model.decoder.block[0].layer[0]

T5LayerSelfAttention(
  (SelfAttention): T5Attention(
    (q): Linear(in_features=768, out_features=768, bias=False)
    (k): Linear(in_features=768, out_features=768, bias=False)
    (v): Linear(in_features=768, out_features=768, bias=False)
    (o): Linear(in_features=768, out_features=768, bias=False)
    (relative_attention_bias): Embedding(32, 12)
  )
  (layer_norm): T5LayerNorm()
  (dropout): Dropout(p=0.1, inplace=False)
)

In [18]:
model.decoder.config.use_cache

True

In [19]:
model.decoder.final_layer_norm

T5LayerNorm()

In [6]:
input_ids = batch['input_ids']  # Batch_size, seq_length
downsampled_features = batch['downsampled_features']  # Batch_size, seq_length, feature_dim
attention_mask = batch['attention_mask']  # Batch_size, seq_length

shifted_left_outputs = torch.cat([input_ids[:, 1:], torch.full((input_ids.size(0), 1), tokenizer.pad_token_id, dtype=torch.long, device=device)], dim=1)
tokenizer.batch_decode(shifted_left_outputs)

['而 对 楼 市 成 交 抑 制 作 用 最 大 的 限 购 [SEP] [PAD] [PAD] [PAD] [PAD]',
 '也 成 为 地 方 政 府 的 眼 中 钉 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '自 六 月 底 呼 和 浩 特 市 率 先 宣 布 取 消 限 购 后 [SEP] [PAD]',
 '各 地 政 府 便 纷 纷 跟 进 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]']

In [7]:
B, C, T, F = downsampled_features.shape
outputs = model(input_ids, attention_mask, downsampled_features.view(B, T, C*F))  # Batch_size, seq_length, vocab_size
outputs.shape

torch.Size([4, 20, 21128])

In [8]:
# Compute CER and loss
ids_prediction = outputs.argmax(dim=-1)  # Batch_size, seq_length
predictions = tokenizer.batch_decode(ids_prediction, skip_special_tokens=True)  # Batch_size
references = tokenizer.batch_decode(shifted_left_outputs, skip_special_tokens=True)  # Batch_size

In [9]:
print(f'References: {references}')

References: ['而 对 楼 市 成 交 抑 制 作 用 最 大 的 限 购', '也 成 为 地 方 政 府 的 眼 中 钉', '自 六 月 底 呼 和 浩 特 市 率 先 宣 布 取 消 限 购 后', '各 地 政 府 便 纷 纷 跟 进']


In [10]:
print(f'Predictions: {predictions}')

Predictions: ['ex 闌 脳 筑晝還綑弈詡 鲑 a灬 helpapp 驍端 笆窗 錳 间ien', '##餡澆 蚤箍奘兰 蜆過崛 辻 畳med 1907锏锏 喟 焘 による 铿 第', '##しています 鈕 fx娄鋅 汶 繩 xddd丐 害将 惘撮訶のは realᅣvr 畳yo', '##琳尻 贻 蒞 捍 co2 淌 电槓 饰骁 飘炼 剣靴淼 hard 间 庠挾']


In [ ]:
cer_score = cer.compute(predictions=predictions, references=references)
cer_score

In [15]:
outputs.shape, shifted_left_outputs.shape

(torch.Size([4, 20, 21128]), torch.Size([4, 20]))

In [31]:
def loss_fn(outputs, labels, cer_score, criterion, gamma=1.0, ignore_index=0):
    """
    Compute the weighted loss for the Decoder using alpha weights based on CER.

    Args:
        outputs (torch.Tensor): Model predictions of shape (batch_size, seq_length, vocab_size).
        labels (torch.Tensor): Ground truth labels of shape (batch_size, seq_length).
        cer_score (float): Character Error Rate (CER) for the Decoder.
        gamma (float): Hyperparameter to control the influence of CER in alpha computation.
        ignore_index (int): The padding value in the labels to be ignored in the loss calculation.

    Returns:
        torch.Tensor: The computed weighted loss.
    """
    # Ensure cer_score is a tensor
    cer_score = torch.tensor(cer_score, dtype=torch.float32, device=outputs.device)

    # Compute alpha weight based on CER
    alpha = -(cer_score ** gamma) * torch.log(1 - cer_score + 1e-8)  # Add epsilon to avoid log(0)

    # Compute the base loss
    base_loss = criterion(outputs.mT, labels)  # Flatten outputs and labels
    if base_loss < 1:
        try: 
            alpha = -(cer_score ** gamma) * torch.log(1 - cer_score + 1e-8)  # Add epsilon to avoid log(0)
            base_loss = base_loss * alpha  # Apply alpha weight to the loss
        except:
            base_loss = base_loss # Avoid cer_score is bigger than 1

    return base_loss

criterion = torch.nn.CrossEntropyLoss(ignore_index=pad_token)
loss = loss_fn(outputs, shifted_left_outputs, cer_score, criterion)
loss

C:\Users\huy\AppData\Local\Temp\ipykernel_16536\4162233661.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cer_score = torch.tensor(cer_score, dtype=torch.float32, device=outputs.device)


tensor(10.0796, device='cuda:0', grad_fn=<NllLoss2DBackward0>)